In [213]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [214]:
! pip install navec
! pip install slovnet

In [215]:
import pandas as pd
import random
import string
import nltk

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
from navec import Navec
from slovnet.model.emb import NavecEmbedding

from tqdm.notebook import tqdm
tqdm.pandas()

# Подгружаем данные

In [216]:
glossary = pd.read_csv('gdrive/MyDrive/AI_HUB_HACK/gloss.csv')
glossary

,Сокращение,Расшифровка
0,лк,личный кабинет
1,БиР,Беременность и роды
2,зп,заработная плата
3,НДФЛ,Налог на доходы физических лиц
4,СТД,срочный трудовой договор
5,ТК,трудовой договор
6,АО,авансовый отчет
7,SLA,сроки
8,ЭЦП,электронная цифровая подпись
9,КР,кадровый резерв


In [217]:
popular_phrases = pd.read_csv('gdrive/MyDrive/AI_HUB_HACK/popular_phrases.csv')
popular_phrases

,Ответ,Вопрос
0,Для оформления обращения в техническую поддерж...,выйти
1,Чат-бот находится в стадии пилотирования и обу...,Уберите этот чат бот он очень мешает
2,Чат-бот находится в стадии пилотирования и обу...,вернуть старый дизайн
3,Для оформления обращения в техническую поддерж...,хочу создать заявку
4,Для оформления обращения в техническую поддерж...,мне нужно создать заявку
5,Чат-бот находится в стадии пилотирования и обу...,бот тупой
6,Чат-бот находится в стадии пилотирования и обу...,бот не помогает
7,Чат-бот находится в стадии пилотирования и обу...,как убрать бота
8,Для оформления обращения в техническую поддерж...,мне сказали создать заявку в поддержке
9,Для оформления обращения в техническую поддерж...,выход


In [218]:
qa_df=pd.read_csv('gdrive/MyDrive/AI_HUB_HACK/new_qa.csv')
qa_df.head(5)

,id,question,content,category
0,1577,Как заказать выпуск МЧД в ТС5?,1. Заходим на портал поддержки https://company...,ЭЦП
1,1225,Сотруднику в ЛК не приходит ссылка на заполнен...,Анкета СБ приходит при первичном трудоустройстве,СБ
2,1560,как получить выплату по уходу за больным родст...,Больничный с кодом 09 (уход за больным членом ...,уход за больным
3,1561,как получить выплату по уходу за больным родст...,Больничный с кодом 09 (уход за больным членом ...,уход за больным
4,1562,Почему нет выплаты от работодателя по больничн...,Больничный с кодом 09 (уход за больным членом ...,уход за больным


# Смотрим на данные

In [219]:
proportions = qa_df['category'].value_counts(dropna=False).reset_index()
print(proportions.head(5), '\n')
print(proportions.tail(10))

           category  count
0                ЛК    481
1         поддержка    284
2            табель    168
3            отпуск    119
4  удаленная работа     98 

         category  count
22       обучение      7
23   доверенность      7
24   командировка      5
25            дмс      3
26        перевод      2
27             СБ      1
28            МЧД      1
29            SED      1
30         Отпуск      1
31  выручай-карта      1


# Конкатим категорию к началу каждого вопроса

In [220]:
qa_df['question'] = qa_df['category'] + " " + qa_df['question']
qa_df['question'] = qa_df['question'].str.lower()

# Кодируемся

In [221]:
def encode_column(df, column_name):
  id_map = {}
  for x in df[column_name]:
    id_map[x] = id_map.get(x, len(id_map))
  df[f'{column_name}_id'] = df[column_name].map(id_map)
  return id_map

In [222]:
id_map_content = encode_column(qa_df, 'content')
id_map_categories = encode_column(qa_df, 'category')

# Оставляем только те ответы, которые встречаются более 4 раз

In [223]:
qa_df['content_count'] = qa_df['content_id'].map(qa_df.groupby('content_id').agg('size'))
qa_df = qa_df.loc[qa_df['content_count'].ge(4)]

In [224]:
# Перекодируем conten_id
id_map_content = encode_column(qa_df, 'content')

<ipython-input-221-3ff210bc371b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{column_name}_id'] = df[column_name].map(id_map)


# Функции для аугменации

In [225]:
def add_or_remove_punctuation(text):
    """Добавление или удаление знаков препинания."""
    # Возможные варианты добавления знаков препинания
    punctuations = [',', '.', '!', '?']
    words = text.split()

    # Добавляем или удаляем знаки препинания
    if random.random() < 0.5:
        # Добавить знак препинания
        position = random.randint(0, len(words) - 1)
        punct = random.choice(punctuations)
        words[position] = words[position] + punct
    else:
        # Удалить знак препинания, если он есть
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()

    return ' '.join(words)

In [226]:
def introduce_typo(text):
    """Создание опечаток в тексте."""
    if not text:
        return text

    words = text.split()
    index = random.randint(0, len(words) - 1)
    word = words[index]

    # Опечатки: замена, пропуск или дублирование символов
    typo_type = random.choice(['swap', 'remove', 'duplicate'])

    if typo_type == 'swap' and len(word) > 1:
        # Меняем местами соседние буквы
        pos = random.randint(0, len(word) - 2)
        word = list(word)
        word[pos], word[pos + 1] = word[pos + 1], word[pos]
        words[index] = ''.join(word)

    elif typo_type == 'remove' and len(word) > 1:
        # Удаляем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos + 1:]

    elif typo_type == 'duplicate':
        # Дублируем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos] + word[pos:]

    return ' '.join(words)

In [227]:
def shuffle_words(text):
    """Перестановка порядка слов."""
    words = text.split()
    if len(words) > 1:
        random.shuffle(words)
    return ' '.join(words)

# Пример аугментации

In [228]:
# Пример использования
text = "всем привет, динозавры"
augmented_text = add_or_remove_punctuation(text)
augmented_text = introduce_typo(augmented_text)
augmented_text = shuffle_words(augmented_text)

print("Оригинальное предложение:", text)
print("Аугментированное предложение:", augmented_text)

Оригинальное предложение: всем привет, динозавры
Аугментированное предложение: динозаврры привет всем


# Расширение датасета

In [243]:
AUG_NUM = 30

def balance_dataset(qa_df):
    # Шаг 1: Найти самый частовстречаемый ответ
    max_count = qa_df['content_id'].value_counts().max()

    # Шаг 2: Сбалансировать выборку ответов
    augmented_data = []
    for content_id, group in qa_df.groupby('content_id'):
        count = len(group)
        augmented_data.extend(group.to_dict('records'))  # Добавляем все исходные строки

        # Если ответ встречается реже, чем самый частовстречаемый, создаем аугментированные копии вопросов
        for _ in range(min(AUG_NUM, max_count - count)):
            row = group.sample(1).iloc[0].to_dict()  # Случайный вопрос из группы
            question = row['question']

            # Применяем несколько аугментаций последовательно
            augmented_question = add_or_remove_punctuation(question)
            augmented_question = introduce_typo(augmented_question)
            augmented_question = shuffle_words(augmented_question)

            # Сохраняем аугментированный вопрос с исходным ответом
            new_row = row.copy()
            new_row['question'] = augmented_question
            augmented_data.append(new_row)

    # Шаг 3: Создать новый сбалансированный датафрейм
    balanced_qa_df = pd.DataFrame(augmented_data)
    return balanced_qa_df

# Navec

In [230]:
path = '/content/gdrive/MyDrive/AI_HUB_HACK/navec_hudlit_v1_12B_500K_300d_100q.tar'  # 51MB
navec = Navec.load(path)  # ~1 sec, ~100MB RAM

emb = NavecEmbedding(navec)

# Функция для преобразования текста в индексы
def text_to_indices(text, navec):
    tokens = text.split()  # Используем пробелы для токенизации
    indices = [navec.vocab.get(token, navec.vocab['<unk>']) for token in tokens]
    return indices

# Функция для получения эмбеддингов
def get_embeddings(text, emb, navec):
    indices = text_to_indices(text, navec)
    if not indices:
        return torch.zeros(1, emb.embedding_dim)  # Возвращаем нулевой тензор для пустых запросов
    input_tensor = torch.tensor(indices)
    embeddings = emb(input_tensor).mean(dim=0)  # Среднее значение по всем токенам
    return embeddings

# Пайплайн предобработки

In [246]:
def process_splitted_data(df, do_aug=True):
  augmented_df = df.copy()
  if do_aug:
    # print('Аугментация:')
    augmented_df = balance_dataset(df)
  # print('Эмбеддинги:')
  augmented_df['embeddings'] = augmented_df['question'].apply(lambda x: get_embeddings(x, emb, navec).tolist())
  augmented_df['embeddings'] = augmented_df['embeddings'].apply(np.array)
  X = np.stack(augmented_df['embeddings'].values)
  y = augmented_df['content_id']
  return X, y

In [ ]:
# на случай если буду использовать другие модельки
def format_data(data):
  if IS_XGBOOST:
    return xgb.DMatrix(data)
  else:
    return data

In [248]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

n_splits = 5
# Настройка кросс-валидации
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Хранение метрик для каждого фолда
train_metrics = {
    'accuracy': [],
    'precision_macro': [],
    'recall_macro': [],
    'f1_macro': [],
    'precision_micro': [],
    'recall_micro': [],
    'f1_micro': [],
}

test_metrics ={
    'accuracy': [],
    'precision_macro': [],
    'recall_macro': [],
    'f1_macro': [],
    'precision_micro': [],
    'recall_micro': [],
    'f1_micro': [],
}
test_metrics_no_aug = {
    'accuracy': [],
    'precision_macro': [],
    'recall_macro': [],
    'f1_macro': [],
    'precision_micro': [],
    'recall_micro': [],
    'f1_micro': [],
}

for train_index, test_index in tqdm(kf.split(qa_df, qa_df['content_id']), total=n_splits):
    X_train, X_test = qa_df.iloc[train_index], qa_df.iloc[test_index]

    # Обработка данных
    X_train_processed, y_train_processed = process_splitted_data(X_train)
    X_test_processed, y_test_processed = process_splitted_data(X_test)
    X_test_no_aug, y_test_no_aug = process_splitted_data(X_test, do_aug=False)

    # Обучение модели логистической регрессии
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train_processed, y_train_processed)

    # Предсказания
    y_pred_train = model.predict(format_data(X_train_processed))
    y_pred_test = model.predict(format_data(X_test_processed))
    y_pred_test_no_aug = model.predict(format_data(X_test_no_aug))

    # Подсчет метрик для обучающей выборки
    train_metrics['accuracy'].append(accuracy_score(y_train_processed, y_pred_train))
    train_metrics['precision_macro'].append(precision_score(y_train_processed, y_pred_train, average='macro'))
    train_metrics['recall_macro'].append(recall_score(y_train_processed, y_pred_train, average='macro'))
    train_metrics['f1_macro'].append(f1_score(y_train_processed, y_pred_train, average='macro'))
    train_metrics['precision_micro'].append(precision_score(y_train_processed, y_pred_train, average='micro'))
    train_metrics['recall_micro'].append(recall_score(y_train_processed, y_pred_train, average='micro'))
    train_metrics['f1_micro'].append(f1_score(y_train_processed, y_pred_train, average='micro'))

    # Подсчет метрик для тестовой выборки
    test_metrics['accuracy'].append(accuracy_score(y_test_processed, y_pred_test))
    test_metrics['precision_macro'].append(precision_score(y_test_processed, y_pred_test, average='macro'))
    test_metrics['recall_macro'].append(recall_score(y_test_processed, y_pred_test, average='macro'))
    test_metrics['f1_macro'].append(f1_score(y_test_processed, y_pred_test, average='macro'))
    test_metrics['precision_micro'].append(precision_score(y_test_processed, y_pred_test, average='micro'))
    test_metrics['recall_micro'].append(recall_score(y_test_processed, y_pred_test, average='micro'))
    test_metrics['f1_micro'].append(f1_score(y_test_processed, y_pred_test, average='micro'))

    # Подсчет метрик для тестовой выборки
    test_metrics_no_aug['accuracy'].append(accuracy_score(y_test_no_aug, y_pred_test_no_aug))
    test_metrics_no_aug['precision_macro'].append(precision_score(y_test_no_aug, y_pred_test_no_aug, average='macro'))
    test_metrics_no_aug['recall_macro'].append(recall_score(y_test_no_aug, y_pred_test_no_aug, average='macro'))
    test_metrics_no_aug['f1_macro'].append(f1_score(y_test_no_aug, y_pred_test_no_aug, average='macro'))
    test_metrics_no_aug['precision_micro'].append(precision_score(y_test_no_aug, y_pred_test_no_aug, average='micro'))
    test_metrics_no_aug['recall_micro'].append(recall_score(y_test_no_aug, y_pred_test_no_aug, average='micro'))
    test_metrics_no_aug['f1_micro'].append(f1_score(y_test_no_aug, y_pred_test_no_aug, average='micro'))

# Вывод средних метрик по всем фолдам
def print_average_metrics(metrics):
    for key in metrics:
        print(f'{key}: {np.mean(metrics[key]):.4f}')

print("Train Metrics:")
print_average_metrics(train_metrics)

print("\nTest Metrics:")
print_average_metrics(test_metrics)

print("\nTest Metrics (test was not augmented):")
print_average_metrics(test_metrics_no_aug)
print('\n')

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

Train Metrics:
accuracy: 0.9695
precision_macro: 0.9755
recall_macro: 0.9710
f1_macro: 0.9725
precision_micro: 0.9695
recall_micro: 0.9695
f1_micro: 0.9695

Test Metrics:
accuracy: 0.6969
precision_macro: 0.7040
recall_macro: 0.6677
f1_macro: 0.6560
precision_micro: 0.6969
recall_micro: 0.6969
f1_micro: 0.6969

Test Metrics (test was not augmented):
accuracy: 0.8394
precision_macro: 0.7105
recall_macro: 0.7323
f1_macro: 0.7045
precision_micro: 0.8394
recall_micro: 0.8394
f1_micro: 0.8394




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Дальше - если тестить без кроссвалидации (для моделек потяжелее чем линейная регрессия)

In [249]:
X_train, X_test, y_train, y_test = train_test_split(qa_df,
                                                    qa_df['content_id'],
                                                    test_size=0.20,
                                                    stratify=qa_df['content_id'],
                                                    random_state=42)

In [250]:
X_train_processed, y_train_processed = process_splitted_data(X_train)
X_test_processed, y_test_processed = process_splitted_data(X_test)
X_test_no_aug, y_test_no_aug = process_splitted_data(X_test, do_aug=False)

# Обучение модели

In [251]:
# Обучение модели логистической регрессии
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_processed, y_train_processed)

LogisticRegression(max_iter=1000, random_state=42)

# Предикты и метрики

In [252]:
def get_metrics(y_true, y_pred):
  # Подсчет метрик
    accuracy = accuracy_score(y_true, y_pred)
    precision_macro = precision_score(y_true, y_pred, average='macro')
    precision_micro = precision_score(y_true, y_pred, average='micro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_micro = f1_score(y_true, y_pred, average='micro')

    # Вывод метрик
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: (macro:{precision_macro:.4f}, micro:{precision_micro:.4f})')
    print(f'Recall: (macro:{recall_macro:.4f}, micro:{recall_micro:.4f})')
    print(f'F1 Score: (macro:{f1_macro:.4f}, micro:{f1_micro:.4f}) \n')

In [253]:
IS_XGBOOST = False

def format_data(data):
  if IS_XGBOOST:
    return xgb.DMatrix(data)
  else:
    return data

y_pred_train = model.predict(format_data(X_train_processed))
y_pred_test = model.predict(format_data(X_test_processed))
y_pred_test_no_aug = model.predict(format_data(X_test_no_aug))

In [254]:
# Train
get_metrics(y_train_processed, y_pred_train)

Accuracy: 0.9714
Precision: (macro:0.9793, micro:0.9714)
Recall: (macro:0.9734, micro:0.9714)
F1 Score: (macro:0.9757, micro:0.9714) 



In [255]:
# Test
get_metrics(y_test_processed, y_pred_test)

Accuracy: 0.7324
Precision: (macro:0.7507, micro:0.7324)
Recall: (macro:0.7254, micro:0.7324)
F1 Score: (macro:0.7161, micro:0.7324) 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [256]:
# Test no augmentation
get_metrics(y_test_no_aug, y_pred_test_no_aug)

Accuracy: 0.8576
Precision: (macro:0.7516, micro:0.8576)
Recall: (macro:0.7452, micro:0.8576)
F1 Score: (macro:0.7317, micro:0.8576) 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
